In [1]:
import torch
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import OneHotEncoder
from transformers import DistilBertTokenizerFast
from sklearn.model_selection import train_test_split
#from transformers import AutoTokenizer, TFBertModel
from transformers import AutoTokenizer, BertModel,BertForSequenceClassification
from transformers import DistilBertForSequenceClassification,Trainer, TrainingArguments

2023-06-28 17:52:14.240757: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-28 17:52:14.997460: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
def data(filename="Doceree-HCP_Train.csv"):
    data =  pd.read_csv(filename,encoding='latin-1')
    if "IS_HCP" in data.columns:
        data = data[data["IS_HCP"].isin([0,1])]
    data["text_info"] = (data["KEYWORDS"].astype(str)+" "+
                         data["USERCITY"].astype(str)+" "+
                         data["DEVICETYPE"].astype(str)+" "+
                         data["CHANNELTYPE"].astype(str)+" "+
                         data["USERAGENT"].astype(str)+" "+
                         data["URL"].astype(str)+" "+
                         data["PLATFORMTYPE"].astype(str)
                        )

    #data["TAXONOMY"].astype(str)
    return data

In [4]:
def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  return tokenizer.encode_plus(
                        input_text,
                        add_special_tokens = True,
                        max_length = 512,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )

In [6]:
train_data = data("Doceree-HCP_Train.csv")
test_data  = data("Doceree-HCP_Test.csv")
print(f"train size: {train_data.shape}")
print(f"test size: {test_data.shape}")

train_texts  = list(train_data["text_info"].values)
train_labels = [int(i) for i in train_data["IS_HCP"].values]

train_labels = OneHotEncoder().fit_transform(np.array(train_labels).reshape(-1,1)).toarray()

train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train size: (113936, 15)
test size: (28493, 13)


In [7]:
#tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
#model = BertModel.from_pretrained("bert-base-uncased")
train_token_id = []
train_attention_masks = []
for sample in train_texts:
    train_encoding_dict = preprocessing(sample, tokenizer)
    train_token_id.append(train_encoding_dict['input_ids']) 
    train_attention_masks.append(train_encoding_dict['attention_mask'])
train_token_id = torch.cat(train_token_id, dim = 0)
train_attention_masks = torch.cat(train_attention_masks, dim = 0)
train_labels = torch.tensor(train_labels)


val_token_id = []
val_attention_masks = []
for sample in val_texts:
    val_encoding_dict = preprocessing(sample, tokenizer)
    val_token_id.append(val_encoding_dict['input_ids']) 
    val_attention_masks.append(val_encoding_dict['attention_mask'])
val_token_id = torch.cat(val_token_id, dim = 0)
val_attention_masks = torch.cat(val_attention_masks, dim = 0)
val_labels = torch.tensor(val_labels)
# train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_)
# val_encodings   = tokenizer(val_texts, truncation=True, padding=True)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/dhananjay/Competition/venv/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [8]:
#train_labels[0:10]

In [9]:
class HCPDataset(torch.utils.data.Dataset):
    def __init__(self, token_id, attention_masks, labels):
        self.token_id = token_id
        self.attention_masks = attention_masks
        self.labels = labels

    def __getitem__(self, idx):
        item={}
        item["input_ids"] = self.token_id[idx]
        item["attention_mask"] = self.attention_masks[idx]
        item['labels'] = self.labels[idx].to(torch.float32)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = HCPDataset(train_token_id, train_attention_masks, train_labels)
val_dataset = HCPDataset(val_token_id, val_attention_masks, val_labels)

In [10]:
#train_dataset.encodings.data["input_ids"][0]

In [11]:
#train_dataset.labels[1]

In [12]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2, output_attentions= False, output_hidden_states = False,)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.weight', 'pre_classifier.

In [13]:
#model = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity")
num_labels = 2,
    

training_args = TrainingArguments(
    output_dir='./results1',          
    num_train_epochs=1,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=64,   
    warmup_steps=500,               
    weight_decay=0.01,
    evaluation_strategy="steps",
    # strength of weight decay
    #logging_dir='./logs1',            
    #logging_steps=1000,
    save_total_limit = 2,
    load_best_model_at_end=False,
    save_strategy = "no"
)


trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=val_dataset             

trainer.train()

/home/dhananjay/Competition/venv/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss
500,0.441900,0.238360
1000,0.181600,0.170450
1500,0.118500,0.073048
2000,0.094300,0.082108
2500,0.071800,0.045861
3000,0.062200,0.046410
3500,0.042200,0.034680
4000,0.045600,0.033316
4500,0.030900,0.033801
5000,0.031000,0.032983


TrainOutput(global_step=5697, training_loss=0.10182869382044381, metrics={'train_runtime': 4397.1166, 'train_samples_per_second': 20.729, 'train_steps_per_second': 1.296, 'total_flos': 1.2074138452697088e+16, 'train_loss': 0.10182869382044381, 'epoch': 1.0})

In [14]:
trainer.evaluate()

{'eval_loss': 0.02367251180112362,
 'eval_runtime': 180.0483,
 'eval_samples_per_second': 126.566,
 'eval_steps_per_second': 1.983,
 'epoch': 1.0}

In [3]:
save_directory = "./HPC_model"

In [5]:
#trainer.save_model(output_dir="HPC_model1")
model.save_pretrained(save_directory)

tokenizer.save_pretrained(save_directory)

('./HPC_model/tokenizer_config.json',
 './HPC_model/special_tokens_map.json',
 './HPC_model/vocab.txt',
 './HPC_model/added_tokens.json',
 './HPC_model/tokenizer.json')

In [2]:
model= DistilBertForSequenceClassification.from_pretrained(save_directory)
tokenizer = DistilBertTokenizerFast.from_pretrained(save_directory)

In [6]:
model.config

DistilBertConfig {
  "_name_or_path": "HPC_model1",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "multi_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.30.2",
  "vocab_size": 30522
}

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#model.to(device)
test_texts =list(test_data["text_info"].values)
batch_size = 64


test_output = []
for i in range(0,len(test_texts),batch_size):
    test_ids = []
    test_attention_mask = []
    for sample in test_texts[i:i+batch_size]:
        encoding = preprocessing(sample, tokenizer)
        test_ids.append(encoding['input_ids']) 
        test_attention_mask.append(encoding['attention_mask'])


   
    test_ids = torch.cat(test_ids, dim = 0)
    test_attention_mask = torch.cat(test_attention_mask, dim = 0)

    # Forward pass, calculate logit predictions
    with torch.no_grad():
        output = model(test_ids.to(device),attention_mask = test_attention_mask.to(device))
        test_output.append(output.logits.cpu().numpy())

In [17]:
test_predict = np.concatenate(test_output)

In [18]:
test_predict_f = np.argmax(test_predict, axis=1)

In [19]:
test_predict_f.shape

(28493,)

In [22]:
test_data["IS_HCP"]= test_predict_f

In [23]:
test_data.head()

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERCITY,USERZIPCODE,USERAGENT,PLATFORMTYPE,CHANNELTYPE,URL,KEYWORDS,text_info,IS_HCP
0,115501,Desktop,2,75.189.231.103,0d5041ff-f0b6-4d1a-9ad7-0a29f7d485b4,Fayetteville,28305.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6...,Online Medical Journal,Website,https://www.clinicaladvisor.com/home/features/...,Family Practice|Drainage|Clinical|Dermatology|...,Family Practice|Drainage|Clinical|Dermatology|...,0
1,115502,Mobile,2,24.101.33.158,c8396dd0-969f-4d99-a40b-b7bb1f516154,Conneaut Lake,16316.0,Mozilla/5.0 (iPhone; CPU iPhone OS 15_6_1 like...,Online Medical Journal,Website,https://www.ophthalmologyadvisor.com/topics/ca...,General|Clinical|Operative|Medicine|Cardiology...,General|Clinical|Operative|Medicine|Cardiology...,1
2,115503,Desktop,2,172.118.216.142,3c97a081-6518-43f8-9f26-369759cfb471,Covina,91724.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.psychiatryadvisor.com/author/tori-...,Abortion|Anxiety Disorders|Apnea|False|Trauma|...,Abortion|Anxiety Disorders|Apnea|False|Trauma|...,0
3,115504,Desktop,7,71.105.120.171,3e2578c8-f794-41af-a38c-c5cfb3c0f014,Brooklyn,11226.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cureus.com/articles/105482-diverti...,Health|Male|Neurological Surgery|Otolaryngolog...,Health|Male|Neurological Surgery|Otolaryngolog...,0
4,115505,Desktop,2,73.82.211.73,ec2ae7ce-6a8c-4156-98a7-07203e60f483,Marietta,30062.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://www.renalandurologynews.com/home/confe...,chronic kidney disease|pain|nephrology|disease...,chronic kidney disease|pain|nephrology|disease...,1


In [24]:
submission = test_data[["ID","IS_HCP"]].copy()

In [25]:
submission.to_csv("Doceree-HCP_Submission.csv", index=False)

In [27]:
submission.head(10)

,ID,IS_HCP
0,115501,0
1,115502,1
2,115503,0
3,115504,0
4,115505,1
5,115506,0
6,115507,0
7,115508,0
8,115509,0
9,115510,0
